# Monte Carlo sensitivity analysis simulation

This notebook shows how execute Monte Carlo simulations to create
datasets used in the sensitivity analysis.

First, let's import the necessary libraries

In [1]:
from rocketpy import Environment, SolidMotor, Rocket, Flight, MonteCarlo, Function
from rocketpy.stochastic import (
    StochasticEnvironment,
    StochasticSolidMotor,
    StochasticRocket,
    StochasticFlight,
    StochasticNoseCone,
    StochasticTail,
    StochasticTrapezoidalFins,
    StochasticParachute,
    StochasticRailButtons,
)
import datetime

## Set Distributions

The Monte Carlo class allows us to express the parameters uncertainty
by specifying a probability distribution. We consider two possibilities: either the
parameter is constant and there is no uncertainty about it, or we propose a normal
distribution and specify its mean and standard deviation. 

In this example, the goal of the sensitivity analysis is to study the rocket, motor, flight and parachute
parameters influence in the flight outputs (e.g. apogee). The dictionary below defines 
the stochastic parameters along with their mean and standard deviation.

In [2]:
analysis_parameters = {
    # Rocket properties
    "rocket_mass": {"mean": 14.426, "std": 0.5},
    "rocket_radius": {"mean": 127 / 2000, "std": 1 / 1000},
    # Motor Properties
    "motors_dry_mass": {"mean": 1.815, "std": 1 / 100},
    "motors_grain_density": {"mean": 1815, "std": 50},
    "motors_total_impulse": {"mean": 6500, "std": 50},
    "motors_burn_out_time": {"mean": 3.9, "std": 0.2},
    "motors_nozzle_radius": {"mean": 33 / 1000, "std": 0.5 / 1000},
    "motors_grain_separation": {"mean": 5 / 1000, "std": 1 / 1000},
    "motors_grain_initial_height": {"mean": 120 / 1000, "std": 1 / 100},
    "motors_grain_initial_inner_radius": {"mean": 15 / 1000, "std": 0.375 / 1000},
    "motors_grain_outer_radius": {"mean": 33 / 1000, "std": 0.375 / 1000},
    # Parachutes
    "parachutes_main_cd_s": {"mean": 10, "std": 0.1},
    "parachutes_main_lag": {"mean": 1.5, "std": 0.1},
    "parachutes_drogue_cd_s": {"mean": 1, "std": 0.07},
    "parachutes_drogue_lag": {"mean": 1.5, "std": 0.2},
    # Flight
    "heading": {"mean": 53, "std": 2},
    "inclination": {"mean": 84.7, "std": 1},
}

## Create Standard Objects


We will first create a standard RocketPy simulation objects (e.g. Environment, SolidMotor, etc.) to then create the Stochastic objects. All
deterministic parameters are set to its values, and the stochastic ones are set to the `mean` value defined in the dictionary above.


In [3]:
# Environment

env = Environment(latitude=32.990254, longitude=-106.974998, elevation=1400)
tomorrow = datetime.date.today() + datetime.timedelta(days=1)
env.set_date((tomorrow.year, tomorrow.month, tomorrow.day, 12))
env.set_atmospheric_model(type="Forecast", file="GFS")

# Motor
motor = SolidMotor(
    thrust_source="../../../data/motors/Cesaroni_M1670.eng",
    dry_mass=analysis_parameters["motors_dry_mass"]["mean"],
    nozzle_radius=analysis_parameters["motors_nozzle_radius"]["mean"],
    grain_density=analysis_parameters["motors_grain_density"]["mean"],
    burn_time=analysis_parameters["motors_burn_out_time"]["mean"],
    grain_outer_radius=analysis_parameters["motors_grain_outer_radius"]["mean"],
    grain_initial_inner_radius=analysis_parameters["motors_grain_initial_inner_radius"]["mean"],
    grain_initial_height=analysis_parameters["motors_grain_initial_height"]["mean"],
    grain_separation=analysis_parameters["motors_grain_separation"]["mean"],
    dry_inertia=(0.125, 0.125, 0.002),
    grain_number=5,
    grains_center_of_mass_position=0.397,
    center_of_dry_mass_position=0.317,
    nozzle_position=0,
    throat_radius=11 / 1000,
    coordinate_system_orientation="nozzle_to_combustion_chamber",
)

# Rocket
rocket = Rocket(
    radius=analysis_parameters["rocket_radius"]["mean"],
    mass=analysis_parameters["rocket_mass"]["mean"],
    inertia=(6.321, 6.321, 0.034),
    power_off_drag="../../../data/calisto/powerOffDragCurve.csv",
    power_on_drag="../../../data/calisto/powerOnDragCurve.csv",
    center_of_mass_without_motor=0,
    coordinate_system_orientation="tail_to_nose",
)

rail_buttons = rocket.set_rail_buttons(
    upper_button_position=0.0818,
    lower_button_position=-0.618,
    angular_position=45,
)

rocket.add_motor(motor, position=-1.255)

nose_cone = rocket.add_nose(length=0.55829, kind="vonKarman", position=1.278)

fin_set = rocket.add_trapezoidal_fins(
    n=4,
    root_chord=0.120,
    tip_chord=0.060,
    span=0.110,
    position=-1.04956,
    cant_angle=0.5,
    airfoil=("../../../data/calisto/NACA0012-radians.csv", "radians"),
)

tail = rocket.add_tail(
    top_radius=0.0635, bottom_radius=0.0435, length=0.060, position=-1.194656
)
Main = rocket.add_parachute(
    "Main",
    cd_s=analysis_parameters["parachutes_main_cd_s"]["mean"],
    lag=analysis_parameters["parachutes_main_lag"]["mean"],
    trigger=800,
    sampling_rate=105,
    noise=(0, 8.3, 0.5),
)

Drogue = rocket.add_parachute(
    "Drogue",
    cd_s=analysis_parameters["parachutes_drogue_cd_s"]["mean"],
    lag=analysis_parameters["parachutes_drogue_lag"]["mean"],
    trigger="apogee",
    sampling_rate=105,
    noise=(0, 8.3, 0.5),
)

# Flight
test_flight = Flight(
    rocket=rocket,
    environment=env,
    rail_length=5,
    inclination=analysis_parameters["inclination"]["mean"],
    heading=analysis_parameters["heading"]["mean"],
)

/home/lprates/Desktop/Work/RocketPy/RocketPy/rocketpy/mathutils/function.py:3125: UserWarning: Extrapolation method set to 'constant' because the linear method is not supported.
  warnings.warn(


## Create Stochastic Objects

For each RocketPy object, we will create a ``Stochastic`` counterpart that extends the initial model, allowing us to define the uncertainties of each input parameter. The uncertainty is set as the `std` of the uncertainty dictionary.

### Stochastic Environment

We create a `StochasticEnvironment` to pass to the Monte Carlo class. Our initial goal
in the sensitivity analysis is to study the influence of motor, rocket, flight 
and parachute parameters in the flight variables. Therefore, the enviroment is kept
constant and equals to the prediction made for tomorrow. Note we do not take into 
account the  uncertainty of the prediction.

In [4]:
stochastic_env = StochasticEnvironment(
    environment=env,
)

stochastic_env.visualize_attributes()

Reporting the attributes of the `StochasticEnvironment` object:

Constant Attributes:
	datum                    SIRGAS2000
	elevation                1471.4660781502985
	gravity                  Function from R1 to R1 : (height (m)) → (gravity (m/s²))
	latitude                 32.990254
	longitude                -106.974998
	timezone                 UTC

Stochastic Attributes:
	wind_velocity_x_factor   1.00000 ± 0.00000 (normal)
	wind_velocity_y_factor   1.00000 ± 0.00000 (normal)


### Motor


We can now create a `StochasticSolidMotor` object to define the uncertainties associated with the motor.

In [5]:
stochastic_motor = StochasticSolidMotor(
    solid_motor=motor,
    dry_mass = analysis_parameters["motors_dry_mass"]["std"],
    grain_density = analysis_parameters["motors_grain_density"]["std"], 
    burn_out_time = analysis_parameters["motors_burn_out_time"]["std"],
    nozzle_radius = analysis_parameters["motors_nozzle_radius"]["std"],
    grain_separation = analysis_parameters["motors_grain_separation"]["std"],
    grain_initial_height = analysis_parameters["motors_grain_initial_height"]["std"],
    grain_initial_inner_radius = analysis_parameters["motors_grain_initial_inner_radius"]["std"],
    grain_outer_radius = analysis_parameters["motors_grain_outer_radius"]["std"],
    total_impulse=(analysis_parameters["motors_total_impulse"]["mean"], 
                   analysis_parameters["motors_total_impulse"]["std"]),
)
stochastic_motor.visualize_attributes()

Reporting the attributes of the `StochasticSolidMotor` object:

Constant Attributes:
	burn_start_time                  0
	center_of_dry_mass_position      0.317
	coordinate_system_orientation    nozzle_to_combustion_chamber
	dry_I_11                         0.125
	dry_I_12                         0
	dry_I_13                         0
	dry_I_22                         0.125
	dry_I_23                         0
	dry_I_33                         0.002
	grain_number                     5
	grains_center_of_mass_position   0.397
	interpolate                      linear
	nozzle_position                  0
	throat_radius                    0.011
	thrust_source                    [[0, 0], [0.055, 100.0], [0.092, 1500.0], [0.1, 2000.0], [0.15, 2200.0], [0.2, 1800.0], [0.5, 1950.0], [1.0, 2034.0], [1.5, 2000.0], [2.0, 1900.0], [2.5, 1760.0], [2.9, 1700.0], [3.0, 1650.0], [3.3, 530.0], [3.4, 350.0], [3.9, 0.0]]

Stochastic Attributes:
	burn_out_time                    3.90000 ± 0.20000 (normal)
	dr

### Rocket


We can now create a `StochasticRocket` object to define the uncertainties associated with the rocket.

In [6]:
stochastic_rocket = StochasticRocket(
    rocket=rocket,
    radius=analysis_parameters["rocket_radius"]["std"],
    mass = analysis_parameters["rocket_mass"]["std"],
)
stochastic_rocket.visualize_attributes()

Reporting the attributes of the `StochasticRocket` object:

Constant Attributes:
	I_11_without_motor              6.321
	I_12_without_motor              0
	I_13_without_motor              0
	I_22_without_motor              6.321
	I_23_without_motor              0
	I_33_without_motor              0.034
	center_of_mass_without_motor    0
	coordinate_system_orientation   tail_to_nose
	power_off_drag                  Function from R1 to R1 : (Mach Number) → (Drag Coefficient with Power Off)
	power_on_drag                   Function from R1 to R1 : (Mach Number) → (Drag Coefficient with Power On)

Stochastic Attributes:
	mass                            14.42600 ± 0.50000 (normal)
	power_off_drag_factor           1.00000 ± 0.00000 (normal)
	power_on_drag_factor            1.00000 ± 0.00000 (normal)
	radius                          0.06350 ± 0.00100 (normal)


The `StochasticRocket` still needs to have its aerodynamic surfaces and parachutes added.
As discussed, we need to set the uncertainties in parachute parameters.

In [7]:
stochastic_nose_cone = StochasticNoseCone(
    nosecone=nose_cone,
)

stochastic_fin_set = StochasticTrapezoidalFins(
    trapezoidal_fins=fin_set,
)

stochastic_tail = StochasticTail(
    tail=tail,
)

stochastic_rail_buttons = StochasticRailButtons(
    rail_buttons=rail_buttons, 
)

stochastic_main = StochasticParachute(
    parachute=Main,
    cd_s=analysis_parameters["parachutes_main_cd_s"]["std"],
    lag=analysis_parameters["parachutes_main_lag"]["std"],
)

stochastic_drogue = StochasticParachute(
    parachute=Drogue,
    cd_s=analysis_parameters["parachutes_drogue_cd_s"]["std"],
    lag=analysis_parameters["parachutes_drogue_lag"]["std"],
)

Then we must add them to our stochastic rocket, much like we do in the normal Rocket.



In [8]:
stochastic_rocket.add_motor(stochastic_motor)
stochastic_rocket.add_nose(stochastic_nose_cone)
stochastic_rocket.add_trapezoidal_fins(stochastic_fin_set,)
stochastic_rocket.add_tail(stochastic_tail)
stochastic_rocket.set_rail_buttons(
    stochastic_rail_buttons
)
stochastic_rocket.add_parachute(stochastic_main)
stochastic_rocket.add_parachute(stochastic_drogue)
stochastic_rocket.visualize_attributes()

Reporting the attributes of the `StochasticRocket` object:

Constant Attributes:
	I_11_without_motor              6.321
	I_12_without_motor              0
	I_13_without_motor              0
	I_22_without_motor              6.321
	I_23_without_motor              0
	I_33_without_motor              0.034
	center_of_mass_without_motor    0
	coordinate_system_orientation   tail_to_nose
	power_off_drag                  Function from R1 to R1 : (Mach Number) → (Drag Coefficient with Power Off)
	power_on_drag                   Function from R1 to R1 : (Mach Number) → (Drag Coefficient with Power On)

Stochastic Attributes:
	mass                            14.42600 ± 0.50000 (normal)
	power_off_drag_factor           1.00000 ± 0.00000 (normal)
	power_on_drag_factor            1.00000 ± 0.00000 (normal)
	radius                          0.06350 ± 0.00100 (normal)



### Flight


The setup is concluded by creating the `StochasticFlight`.

In [9]:
stochastic_flight = StochasticFlight(
    flight=test_flight,
    inclination=analysis_parameters["inclination"]["std"],
    heading=analysis_parameters["heading"]["std"],
)
stochastic_flight.visualize_attributes()

Reporting the attributes of the `StochasticFlight` object:

Constant Attributes:
	rail_length           5

Stochastic Attributes:
	heading               53.00000 ± 2.00000 (normal)
	inclination           84.70000 ± 1.00000 (normal)


### Run the Monte Carlo Simulations


Finally, we simulate our flights and save the data.

In [10]:
test_dispersion = MonteCarlo(
    filename="monte_carlo_analysis_outputs/sensitivity_analysis_data",
    environment=stochastic_env,
    rocket=stochastic_rocket,
    flight=stochastic_flight,
)
test_dispersion.simulate(number_of_simulations=100, append=False)

/home/lprates/Desktop/Work/RocketPy/RocketPy/rocketpy/simulation/monte_carlo.py:112: UserWarning: This class is still under testing and some attributes may be changed in next versions
  warnings.warn(


The following input file was imported: monte_carlo_analysis_outputs/sensitivity_analysis_data.inputs.txt
A total of 100 simulations results were loaded from the following output file: monte_carlo_analysis_outputs/sensitivity_analysis_data.outputs.txt

The following error file was imported: monte_carlo_analysis_outputs/sensitivity_analysis_data.errors.txt
Completed 100 iterations. Total CPU time: 72.6 s. Total wall time: 72.6 sed time left: 0 s 
Saving results.  
Results saved to monte_carlo_analysis_outputs/sensitivity_analysis_data.outputs.txt


We give a last check on the variables summary results.

In [92]:
test_dispersion.prints.all()

Monte Carlo Simulation by RocketPy
Data Source:  monte_carlo_analysis_outputs/sensitivity_analysis_data
Number of simulations:  100
Results: 

                Parameter            Mean       Std. Dev.
------------------------------------------------------------
     frontal_surface_wind          -0.993           0.094
 initial_stability_margin           2.215           0.096
out_of_rail_stability_margin           2.274           0.092
                 x_impact        1243.893         112.465
                  t_final         311.568          12.063
         out_of_rail_time           0.348           0.014
                 apogee_y         669.619          78.851
          max_mach_number           0.908           0.031
     out_of_rail_velocity          26.703           1.009
     lateral_surface_wind          -2.783           0.034
                   apogee        5203.714         125.524
                 y_impact         213.037         100.342
                 apogee_x         494.2